In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.8.0


In [3]:
from tensorflow import keras
from keras import layers

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
train_df = pd.read_csv('/content/drive/My Drive/data/Raman_DNA/train.csv',sep=',')
submission_df = pd.read_csv('/content/drive/My Drive/data/Raman_DNA/sample_submission.csv',sep=',')
test_df = pd.read_csv('/content/drive/My Drive/data/Raman_DNA/test.csv',sep=',')

In [8]:
train_df.head()

,row_id,A0T0G0C10,A0T0G1C9,A0T0G2C8,A0T0G3C7,A0T0G4C6,A0T0G5C5,A0T0G6C4,A0T0G7C3,A0T0G8C2,A0T0G9C1,A0T0G10C0,A0T1G0C9,A0T1G1C8,A0T1G2C7,A0T1G3C6,A0T1G4C5,A0T1G5C4,A0T1G6C3,A0T1G7C2,A0T1G8C1,A0T1G9C0,A0T2G0C8,A0T2G1C7,A0T2G2C6,A0T2G3C5,A0T2G4C4,A0T2G5C3,A0T2G6C2,A0T2G7C1,A0T2G8C0,A0T3G0C7,A0T3G1C6,A0T3G2C5,A0T3G3C4,A0T3G4C3,A0T3G5C2,A0T3G6C1,A0T3G7C0,A0T4G0C6,...,A5T3G2C0,A5T4G0C1,A5T4G1C0,A5T5G0C0,A6T0G0C4,A6T0G1C3,A6T0G2C2,A6T0G3C1,A6T0G4C0,A6T1G0C3,A6T1G1C2,A6T1G2C1,A6T1G3C0,A6T2G0C2,A6T2G1C1,A6T2G2C0,A6T3G0C1,A6T3G1C0,A6T4G0C0,A7T0G0C3,A7T0G1C2,A7T0G2C1,A7T0G3C0,A7T1G0C2,A7T1G1C1,A7T1G2C0,A7T2G0C1,A7T2G1C0,A7T3G0C0,A8T0G0C2,A8T0G1C1,A8T0G2C0,A8T1G0C1,A8T1G1C0,A8T2G0C0,A9T0G0C1,A9T0G1C0,A9T1G0C0,A10T0G0C0,target
0,0,-9.536743e-07,-0.000010,-0.000043,-0.000114,-0.000200,-0.000240,-0.000200,-0.000114,-0.000043,-0.000010,-9.536743e-07,-0.000010,-8.583069e-05,-0.000343,-0.000801,-0.001202,-0.001202,-0.000801,-0.000343,-0.000086,-0.000010,-4.291534e-05,-0.000343,-0.001202,-0.002403,-0.003004,-0.002403,-0.001202,-0.000343,-0.000043,-0.000114,-0.000801,-0.002403,-0.004005,-0.004005,-0.002403,-0.000801,-0.000114,-0.000200,...,-0.002403,0.008798,-0.001202,0.009760,-0.000200,-0.000801,-0.001202,-0.000801,-0.000200,-0.000801,-0.002403,-0.002403,-0.000801,-0.001202,-0.002403,-0.001202,-0.000801,-0.000801,-0.000200,-0.000114,-0.000343,-0.000343,-0.000114,-0.000343,-0.000687,-0.000343,-0.000343,-0.000343,-0.000114,-0.000043,-0.000086,-0.000043,-0.000086,-0.000086,-0.000043,-0.000010,-0.000010,-0.000010,-9.536743e-07,Streptococcus_pyogenes
1,1,-9.536743e-07,-0.000010,-0.000043,0.000886,-0.000200,0.000760,-0.000200,-0.000114,-0.000043,-0.000010,-9.536743e-07,-0.000010,-8.583069e-05,-0.000343,0.000199,-0.000202,0.001798,-0.000801,-0.000343,-0.000086,-0.000010,-4.291534e-05,-0.000343,0.001798,-0.000403,0.001996,0.003597,-0.001202,-0.000343,-0.000043,-0.000114,-0.000801,-0.000403,0.002995,0.002995,0.000597,0.001199,-0.000114,-0.000200,...,0.000597,0.002798,0.001798,0.000760,-0.000200,-0.000801,-0.000202,-0.000801,0.000800,0.000199,0.003597,0.001597,-0.000801,-0.000202,-0.001403,0.000798,0.001199,0.001199,0.000800,-0.000114,-0.000343,-0.000343,-0.000114,-0.000343,0.001313,0.000657,0.001657,0.001657,0.000886,-0.000043,-0.000086,-0.000043,0.000914,0.000914,-0.000043,-0.000010,-0.000010,-0.000010,-9.536743e-07,Salmonella_enterica
2,2,-9.536743e-07,-0.000002,0.000007,0.000129,0.000268,0.000270,0.000243,0.000125,0.000001,-0.000007,4.632568e-08,-0.000004,1.693115e-07,0.000008,0.000366,0.000766,0.000937,0.000632,0.000101,-0.000021,-0.000008,8.465576e-08,-0.000080,0.000114,0.000710,0.001307,0.001065,0.000375,-0.000017,-0.000017,-0.000014,-0.000094,0.000183,0.000817,0.000968,0.000469,-0.000007,-0.000017,-0.000031,...,-0.000031,0.000146,0.000118,0.000156,0.000003,0.000080,0.000219,0.000225,0.000017,0.000014,0.000319,0.000355,0.000175,0.000030,0.000347,0.000229,0.000203,0.000238,0.000120,0.000027,0.000097,0.000159,0.000070,0.000138,0.000315,0.000214,0.000178,0.000296,0.000118,0.000042,0.000084,0.000048,0.000081,0.000106,0.000072,0.000010,0.000008,0.000019,1.046326e-06,Salmonella_enterica
3,3,4.632568e-08,-0.000006,0.000012,0.000245,0.000492,0.000522,0.000396,0.000197,-0.000003,-0.000007,-9.536743e-07,-0.000010,-1.783069e-05,0.000068,0.000722,0.001594,0.001913,0.001111,0.000247,-0.000047,-0.000010,-1.891534e-05,-0.000138,0.000161,0.001345,0.002526,0.002155,0.000754,-0.000029,-0.000025,-0.000034,-0.000179,0.000345,0.001427,0.001964,0.000861,-0.000038,-0.000050,-0.000067,...,-0.000075,0.000284,0.000251,0.000296,-0.000020,0.000194,0.000399,0.000407,0.000040,0.000058,0.000505,0.000784,0.000296,0.000128,0.000835,0.000445,0.000401,0.000412,0.000246,0.000047,0.000160,0.000309,0.000129,0.000243,0.000597,0.000428,0.000363,0.000472,0.000197,0.000068,0.000151,0.000100,0.000180,0.000202,0.000153,0.000021,0.000015,0.000046,-9.536743e-07,Salmonella_enterica
4,4,-9.536743e-07,-0.000010,-0.000043,-0.000114,-0.000200,-0.000240,-0.000200,-0.000114,-0.000043,-0.000010,-9.536

In [9]:
test_df.head()

,row_id,A0T0G0C10,A0T0G1C9,A0T0G2C8,A0T0G3C7,A0T0G4C6,A0T0G5C5,A0T0G6C4,A0T0G7C3,A0T0G8C2,A0T0G9C1,A0T0G10C0,A0T1G0C9,A0T1G1C8,A0T1G2C7,A0T1G3C6,A0T1G4C5,A0T1G5C4,A0T1G6C3,A0T1G7C2,A0T1G8C1,A0T1G9C0,A0T2G0C8,A0T2G1C7,A0T2G2C6,A0T2G3C5,A0T2G4C4,A0T2G5C3,A0T2G6C2,A0T2G7C1,A0T2G8C0,A0T3G0C7,A0T3G1C6,A0T3G2C5,A0T3G3C4,A0T3G4C3,A0T3G5C2,A0T3G6C1,A0T3G7C0,A0T4G0C6,...,A5T3G1C1,A5T3G2C0,A5T4G0C1,A5T4G1C0,A5T5G0C0,A6T0G0C4,A6T0G1C3,A6T0G2C2,A6T0G3C1,A6T0G4C0,A6T1G0C3,A6T1G1C2,A6T1G2C1,A6T1G3C0,A6T2G0C2,A6T2G1C1,A6T2G2C0,A6T3G0C1,A6T3G1C0,A6T4G0C0,A7T0G0C3,A7T0G1C2,A7T0G2C1,A7T0G3C0,A7T1G0C2,A7T1G1C1,A7T1G2C0,A7T2G0C1,A7T2G1C0,A7T3G0C0,A8T0G0C2,A8T0G1C1,A8T0G2C0,A8T1G0C1,A8T1G1C0,A8T2G0C0,A9T0G0C1,A9T0G1C0,A9T1G0C0,A10T0G0C0
0,200000,-9.536743e-07,-0.000002,-9.153442e-07,0.000024,0.000034,-0.000002,0.000021,0.000024,-0.000009,-0.000008,-9.536743e-07,-0.000003,-0.000017,-0.000074,0.000038,0.000068,0.000164,0.000180,-0.000003,-0.000025,-0.000009,0.000004,-0.000101,-0.000171,0.000006,0.000330,0.000580,0.000228,-0.000015,-0.000016,-0.000026,-0.000146,-0.000075,0.000212,0.000565,0.000359,0.000072,-0.000005,-0.000043,...,0.000163,0.000105,0.000178,0.000151,0.000156,5.572839e-05,0.000258,0.000321,0.000204,0.000006,0.000180,0.000540,0.000490,0.000174,0.000175,0.000447,0.000262,0.000279,0.000254,0.000111,0.000043,0.000150,0.000192,0.000076,0.000191,0.000387,0.000214,0.000184,0.000268,0.000121,0.000039,0.000085,0.000055,0.000108,0.000090,0.000059,0.000010,0.000006,0.000027,4.632568e-08
1,200001,-9.536743e-07,-0.000010,-4.291534e-05,-0.000114,0.001800,-0.000240,0.001800,-0.000114,0.000957,-0.000010,-9.536743e-07,-0.000010,-0.000086,-0.000343,0.001199,0.001798,0.000798,0.000199,0.000657,-0.000086,-0.000010,-0.000043,-0.000343,-0.001202,0.001597,0.002996,-0.002403,0.000798,-0.000343,-0.000043,-0.000114,-0.000801,0.003597,0.001995,0.002995,-0.000403,0.001199,-0.000114,-0.000200,...,-0.002807,0.000597,0.000798,0.001798,0.000760,-2.002716e-04,0.000199,0.001798,0.000199,-0.000200,-0.000801,0.000597,-0.000403,0.000199,-0.001202,0.000597,0.001798,0.000199,0.001199,-0.000200,-0.000114,-0.000343,-0.000343,-0.000114,-0.000343,-0.000687,0.002657,-0.000343,-0.000343,-0.000114,-0.000043,0.000914,-0.000043,-0.000086,-0.000086,-0.000043,-0.000010,-0.000010,-0.000010,-9.536743e-07
2,200002,4.632568e-08,0.000003,8.465576e-08,-0.000014,0.000007,-0.000005,-0.000004,0.000003,0.000004,-0.000008,-9.536743e-07,-0.000003,-0.000003,-0.000045,-0.000038,-0.000110,-0.000127,-0.000076,-0.000012,-0.000004,-0.000005,0.000009,-0.000052,-0.000110,-0.000280,-0.000220,-0.000165,-0.000048,-0.000023,-0.000005,-0.000006,-0.000073,-0.000154,-0.000332,-0.000287,-0.000172,-0.000035,0.000006,-0.000007,...,0.000699,0.000423,0.000300,0.000281,0.000104,3.172839e-05,0.000097,0.000247,0.000162,0.000073,0.000156,0.000623,0.000478,0.000213,0.000264,0.000560,0.000420,0.000304,0.000329,0.000112,0.000039,0.000142,0.000155,0.000080,0.000177,0.000428,0.000250,0.000251,0.000338,0.000122,0.000041,0.000102,0.000084,0.000111,0.000117,0.000070,0.000020,0.000030,0.000021,4.632568e-08
3,200003,-9.536743e-07,-0.000008,8.084656e-06,0.000216,0.000420,0.000514,0.000452,0.000187,-0.000005,-0.000008,4.632568e-08,-0.000006,-0.000024,0.000060,0.000712,0.001579,0.001887,0.001299,0.000203,-0.000041,-0.000009,-0.000016,-0.000128,0.000128,0.001283,0.002351,0.002028,0.000670,-0.000037,-0.000025,-0.000063,-0.000171,0.000338,0.001279,0.001807,0.000838,0.000039,-0.000053,-0.000068,...,-0.000282,-0.000026,0.000226,0.000190,0.000284,7.283936e-07,0.000168,0.000405,0.000378,0.000037,0.000044,0.000462,0.000797,0.000306,0.000091,0.000766,0.000449,0.000417,0.000425,0.000222,0.000049,0.000185,0.000304,0.000134,0.000204,0.000635,0.000431,0.000317,0.000460,0.000215,0.000069,0.000158,0.000098,0.000175,0.000217,0.000150,0.000018,0.000016,0.000051,-9.536743e-07
4,200004,-9.536743e-07,-0.000010,-4.291534e-05,-0.000114,-0.000200,-0.000240,-0.000200,-0.000114,-0.000043,-0.000010,-9.536743e-07,-0.000010,-0.000086,-0.000343,-0.000801,-0.001

In [10]:
print('Missing Values in Train data: ', train_df.isna().any().sum())
print('Missing Values in Test data: ',test_df.isna().any().sum())

Missing Values in Train data:  0
Missing Values in Test data:  0


In [11]:
train_df.target.unique()

array(['Streptococcus_pyogenes', 'Salmonella_enterica',
       'Enterococcus_hirae', 'Escherichia_coli', 'Campylobacter_jejuni',
       'Streptococcus_pneumoniae', 'Staphylococcus_aureus',
       'Escherichia_fergusonii', 'Bacteroides_fragilis',
       'Klebsiella_pneumoniae'], dtype=object)

In [12]:
train_df.shape

(200000, 288)

In [13]:
X = train_df.copy()
y = X.pop('target')

In [ ]:
y_train = train_df['target']
X_train = train_df.drop(['row_id','target'], axis=1)

In [15]:
# encode class values as integers
le = LabelEncoder()
y_train = le.fit_transform(y_train)
le.classes_

array(['Bacteroides_fragilis', 'Campylobacter_jejuni',
       'Enterococcus_hirae', 'Escherichia_coli', 'Escherichia_fergusonii',
       'Klebsiella_pneumoniae', 'Salmonella_enterica',
       'Staphylococcus_aureus', 'Streptococcus_pneumoniae',
       'Streptococcus_pyogenes'], dtype=object)

In [31]:
y_train

6167          Campylobacter_jejuni
28869        Klebsiella_pneumoniae
64376       Streptococcus_pyogenes
165551       Klebsiella_pneumoniae
59522         Campylobacter_jejuni
                    ...           
172057      Escherichia_fergusonii
99709        Staphylococcus_aureus
182127          Enterococcus_hirae
119942        Bacteroides_fragilis
27489     Streptococcus_pneumoniae
Name: target, Length: 150000, dtype: object

In [25]:
train_df.drop_duplicates(inplace = True)

In [24]:
y_train

6167          Campylobacter_jejuni
28869        Klebsiella_pneumoniae
64376       Streptococcus_pyogenes
165551       Klebsiella_pneumoniae
59522         Campylobacter_jejuni
                    ...           
172057      Escherichia_fergusonii
99709        Staphylococcus_aureus
182127          Enterococcus_hirae
119942        Bacteroides_fragilis
27489     Streptococcus_pneumoniae
Name: target, Length: 150000, dtype: object

In [28]:
X.shape

(200000, 287)

In [30]:
y = le.transform(.target)

0    Streptococcus_pyogenes
1       Salmonella_enterica
2       Salmonella_enterica
3       Salmonella_enterica
4        Enterococcus_hirae
Name: target, dtype: object

In [32]:
y = le.transform(train_df.target)

In [35]:
y.shape

(200000,)

In [36]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [37]:
model = RandomForestRegressor(n_estimators=30, min_samples_leaf=5)

In [41]:
test_df.head()
X_test = test_df.drop('row_id', axis=1)

In [ ]:
model.fit(X, y)

In [ ]:
np.sqrt(mean_squared_error(y, model.predict(X_)))

In [42]:
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis = 1)
y_pred = le.inverse_transform(y_pred)
y_pred

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names seen at fit time, yet now missing:
- row_id

  warnings.warn(message, FutureWarning)


ValueError: ignored

,row_id,target
0,200000,Salmonella_enterica
1,200001,Salmonella_enterica
2,200002,Salmonella_enterica
3,200003,Salmonella_enterica
4,200004,Salmonella_enterica
...,...,...
99995,299995,Salmonella_enterica
99996,299996,Salmonella_enterica
99997,299997,Salmonella_enterica
99998,299998,Salmonella_enterica


In [39]:

submission_df['target'] = y_pred
submission_df.to_csv('/content/drive/My Drive/data/Raman_DNA/sample_submission_2.csv', index=False)
